<hr style="border:2px solid coral"></hr>

## Practice : Iterative solvers


<hr style="border:2px solid coral"></hr>


In [2]:
from matplotlib.pyplot import *
from numpy import *

### Example : 

Use the Jacobi iteration 

\begin{equation}
\mathbf x_{k+1} = \mathbf x_k + D^{-1}(\mathbf b - A\mathbf x_k)
\end{equation}

to solve the linear system $A \mathbf x = \mathbf b$, where

\begin{equation}
A = 
\begin{bmatrix}
1 & 4 & -2 \\
0 & 2 & 3 \\
-1 & 0 & 5
\end{bmatrix}, \qquad \mathbf b = 
\begin{bmatrix}
1 \\ 2 \\ -3
\end{bmatrix}
\end{equation}

Use 
\begin{equation}
\Vert \mathbf x_{k+1} - \mathbf x_k \Vert_\infty < \tau = 10^{-12}
\end{equation}

as stopping criteria. 

* Report the inf-norm of the error and residual. 



#### Tips

* You can form the matrix $A$ explicitly and compute the matrix vector multiply as `A@x`.   However, do not form $D$ explicitly.  


* Recall that the inf-norm $\Vert \cdot \Vert_\infty$ of a vector $\mathbf v$ is given by 

\begin{equation}
\Vert \mathbf v \Vert_\infty = \max_{i} |v_i|
\end{equation}

* If the iteration doesn't converge, why not?  Modify the matrix so the system converges

<hr style="border:1px solid coral"></hr>
<center>
<span style="font-weight:bold; font-size:16px"> See how far you get without referring to any notes.</span>
</center>
<hr style="border:1px solid coral"></hr>


In [3]:
from numpy.linalg import norm, solve

A = array([[7,4,-2],[0,6,3],[-1,0,5]],dtype = float64)
b = array([[1],[2],[-3]],dtype=float64)
print("A = ")
print(A)
print("")

print("b = ")
print(b)
print("")

D = diag(A).reshape((3,1))

kmax = 100
tol= 1e-6
# TODO : Iteration to compute solution. 

xk = zeros((3,1))
for k in range(kmax):
    zk = (b - A@xk)/D
    xkp1 = xk + zk
    err = norm(zk,inf)
    print(f"{k:5d} {err:24.4e}")
    if err < tol:
        break
    xk = xkp1
        


# TODO : Report error and residual in your solution.
res = norm(b - A@xk,inf)

print("")
print(f"Residual = {res:12.4e}")

print("")
with printoptions(formatter={'float' : "{:20.16f}".format}):
    print(xk)

# norm(v,inf) == max(abs(v))

A = 
[[ 7.  4. -2.]
 [ 0.  6.  3.]
 [-1.  0.  5.]]

b = 
[[ 1.]
 [ 2.]
 [-3.]]

    0               6.0000e-01
    1               3.6190e-01
    2               1.6327e-01
    3               3.6190e-02
    4               3.0010e-02
    5               1.0045e-02
    6               3.0010e-03
    7               2.2888e-03
    8               7.1285e-04
    9               2.6965e-04
   10               1.7152e-04
   11               5.6143e-05
   12               2.5210e-05
   13               1.3010e-05
   14               4.6487e-06
   15               2.1840e-06
   16               1.0090e-06
   17               3.9044e-07

Residual =   2.7331e-06

[[ -0.4408595031324349]
 [  0.6774189643443280]
 [ -0.6881716988177394]]


### Jacobi : C Version 
<hr style="border:2px solid black"></hr>

Below, we solve the above matrix problem using C. 

In [35]:
%%file jacobi_01.c

#include <stdio.h>
#include <stdlib.h>

#include <math.h>

void matvec(double x[3], double Ax[3])
{          
    double A[9] = {7,4,-2,0,6,3,-1,0,5};
    
    Ax[0] = A[0]*x[0] + A[1]*x[1] + A[2]*x[2];
    Ax[1] = A[3]*x[0] + A[4]*x[1] + A[5]*x[2];
    Ax[2] = A[6]*x[0] + A[7]*x[1] + A[8]*x[2];
}

void jacobi(double b[3], double x[3], double tol, int kmax, int prt)
{    
    double Ax[3];
    double xkp1[3];

    double D[3];
    
    // # Trick to get the diagonal
    double xk[3] = {0,0,0};    
    for(int j = 0; j < 3; j++)
    {
        xk[j] = 1;
        matvec(xk,Ax);
        D[j] = Ax[j];
        xk[j] = 0;
    }
    
    for(int k = 0; k < kmax; k++)
    {
        matvec(xk,Ax);
        
        double err = 0;
        for(int i = 0; i < 3; i++)
        {
            double r = b[i] - Ax[i];
            double dstep = r/D[i];
            xkp1[i] = xk[i] + dstep;  
            err = fabs(dstep) > err ? fabs(dstep) : err;
        }
        printf("%5d %12.4e\n",k,err);
        
        // Parallel reduce to get err. 
        
        
        if (err < tol)
            break;
        
        for(int i = 0; i < 3; i++)
            xk[i] = xkp1[i];        
    }
    for(int i = 0; i < 3; i++)
        x[i] = xkp1[i];        
}

int main(int argc, char** argv)
{    
    int N = 3;
    double tol = 1e-12;
    int kmax = 1000;
    int prt = 1;
        
    double x[3] = {0,0,0};
    double b[3] = {1,2,-3};    
    
    jacobi(b,x,tol,kmax,prt);

    double maxres = 0;

    printf("\n");
    double Ax[3];
    matvec(x,Ax);
    for(int i = 0; i < 3; i++)
    {
        printf("x[%d] = %24.16f\n",i,x[i]);
        
        double r = fabs(b[i] - Ax[i]);
        maxres = r > maxres ? r : maxres;        
    }
    
    printf("\n");
    printf("Residual : %12.4e\n",maxres);
    
    return 0;
}    

Overwriting jacobi_01.c


In [36]:
%%bash

rm -rf jacobi_01.o jacobi_01

gcc -o jacobi_01 jacobi_01.c

jacobi_01

    0   6.0000e-01
    1   3.6190e-01
    2   1.6327e-01
    3   3.6190e-02
    4   3.0010e-02
    5   1.0045e-02
    6   3.0010e-03
    7   2.2888e-03
    8   7.1285e-04
    9   2.6965e-04
   10   1.7152e-04
   11   5.6143e-05
   12   2.5210e-05
   13   1.3010e-05
   14   4.6487e-06
   15   2.1840e-06
   16   1.0090e-06
   17   3.9044e-07
   18   1.8246e-07
   19   7.9971e-08
   20   3.2737e-08
   21   1.4996e-08
   22   6.4404e-09
   23   2.7276e-09
   24   1.2249e-09
   25   5.2389e-10
   26   2.2586e-10
   27   9.9933e-11
   28   4.2843e-11
   29   1.8617e-11
   30   8.1587e-12
   31   3.5120e-12
   32   1.5301e-12
   33   6.6686e-13

x[0] =      -0.4408602150532534
x[1] =       0.6774193548384390
x[2] =      -0.6881720430105174

Residual :   2.0166e-12


Compare results to Python results from above : 

    Residual =   4.6680e-12

    [[ -0.4408602150525865]
     [  0.6774193548380878]
     [ -0.6881720430102113]]

<hr style="border:2px solid coral"></hr>

## Example : Elliptic problem (Matrix-free Jacobi)

<hr style="border:2px solid coral"></hr>


Use the matrix-free Jacobi method to solve the 1d elliptic problem 

\begin{equation}
u''(x) = \sin(x), \quad x \in [0,\pi]
\end{equation}

subject to boundary conditions $u(0) = u(1) = 0$.   

* Discretize the equation using the centered finite difference scheme at nodes. 

* Solve the problem on a mesh of size $N=8$, where $N$ is the number of mesh cells.  

* To check your answer, use the true solution $u(x) = -sin(x)$. 


### Example : Matrix-free Jacobi (parallel version)

<hr style="border:2px black solid"></hr>

What changes need to be made to the serial code to run the code in parallel? 